### Directory

In [1]:
from pathlib import Path
import os
import multiprocessing

NUM_PROCESSORS=multiprocessing.cpu_count()
print("Cpu count: ",NUM_PROCESSORS)

DIR='./Dataset/'

if os.uname()[1].find('gilbreth')==0: ##if not darwin(mac/locallaptop)
    DIR='/scratch/gilbreth/das90/Dataset/'
elif os.uname()[1].find('unimodular')==0:
    DIR='/scratch2/das90/Dataset/'
elif os.uname()[1].find('Siddharthas')==0:
    DIR='/Users/siddharthashankardas/Purdue/Dataset/'  
else:
    DIR='./Dataset/'
    
Path(DIR).mkdir(parents=True, exist_ok=True)

RESULTS_DIR=DIR+'RESULTS/'
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)

print("Data directory: ", DIR)
print("Result directory:", RESULTS_DIR)

Cpu count:  20
Data directory:  /scratch/gilbreth/das90/Dataset/
Result directory: /scratch/gilbreth/das90/Dataset/RESULTS/


In [2]:
import random        
random.seed(12345)

import numpy as np
np.random.seed(12345)

## Dataset

In [3]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.datasets import Reddit, Reddit2, Flickr, Yelp, AmazonProducts, PPI,  OGB_MAG,  FakeDataset


In [4]:
# import numpy as np
# np.load.__defaults__=(None, True, True, 'ASCII')
# # old = np.load
# # np.load = lambda *a,**k: old(*a,allow_pickle=True)

In [5]:
#"Cora", "CiteSeer", "PubMed", "Reddit", "Flickr", 
#"Yelp", "AmazonProducts","Reddit2"," OGB_MAG"
#"Fake"
#"Moon"

In [6]:
DATASET_NAME='Cora' 

def get_data(DATASET_NAME='Cora'):

    if DATASET_NAME in ["Cora", "CiteSeer", "PubMed"]:
        dataset = Planetoid(root=DIR+'Planetoid', name=DATASET_NAME, transform=NormalizeFeatures())

    elif DATASET_NAME == "Reddit2":
        dataset = Reddit2(root=DIR+'Reddit2', transform=NormalizeFeatures())

    elif DATASET_NAME == "Reddit":
        dataset = Reddit(root=DIR+'Reddit', transform=NormalizeFeatures())
        
    elif DATASET_NAME == "Flickr":
        dataset = Flickr(root=DIR+'Flickr', transform=NormalizeFeatures())
    
    elif DATASET_NAME == "Yelp":
        dataset = Yelp(root=DIR+'Yelp', transform=NormalizeFeatures())
    
    elif DATASET_NAME == "AmazonProducts":
        dataset = AmazonProducts(root=DIR+'AmazonProducts', transform=NormalizeFeatures())
    
    elif DATASET_NAME == "OGB_MAG":
        dataset = OGB_MAG(root=DIR+'OGB_MAG', preprocess='metapath2vec', transform=NormalizeFeatures())
        
    elif DATASET_NAME == "Fake":
        import torch        
        
        dataset = FakeDataset(num_graphs = 1, 
                              avg_num_nodes = 1000, 
                              avg_degree = 10, 
                              num_channels = 64, 
                              edge_dim = 0, 
                              num_classes = 10, 
                              task = 'auto', 
                              is_undirected = True,                               
                              transform=NormalizeFeatures())
        
        data = dataset[0]
        train=0.3;val=0.3;test=1-train-val

        l=len(data.y)
        indexs=list(range(l))
        random.shuffle(indexs)

        train_index=indexs[:int(train*l)]
        val_index=indexs[int(train*l):int(train*l)+int(val*l)]
        test_index=indexs[int(train*l)+int(val*l):]

        data.train_mask=torch.zeros(l, dtype=torch.bool)
        data.train_mask[train_index]=True
        data.val_mask=torch.zeros(l, dtype=torch.bool)
        data.val_mask[val_index]=True
        data.test_mask=torch.zeros(l, dtype=torch.bool)
        data.test_mask[test_index]=True
        
        dataset.root=DIR+'FakeDataset'
        
        print(data)
        
        return data, dataset
        
    
    elif DATASET_NAME=='Moon':
        import ipynb.fs.full.utils.MoonGraph as GMoon        
        dataset = GMoon.MoonDataset(n_samples=1000, degree=10)    
        G, data =dataset[0]    
        #GMoon.drawMoon(G,data)
   
    else:    
        raise Exception('dataset not found')

    
    if DATASET_NAME == "OGB_MAG":
        print(f'Dataset: {dataset}:')
        print('======================')
        print(f'Number of graphs: {len(dataset)}')
        
        data = dataset[0]
    
    else:
        print()
        print(f'Dataset: {dataset}:')
        print('======================')
        print(f'Number of graphs: {len(dataset)}')
        print(f'Number of features: {dataset.num_features}')
        print(f'Number of classes: {dataset.num_classes}')
        
        if DATASET_NAME in ['Moon']:
            None
        else:
            data = dataset[0]  # Get the first graph object.

        print()
        print(data)
        print('===========================================================================================================')

        # Gather some statistics about the graph.
        print(f'Number of nodes: {data.num_nodes}')
        print(f'Number of edges: {data.num_edges}')
        print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
        print(f'Number of training nodes: {data.train_mask.sum()}')
        print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
        print(f'Has isolated nodes: {data.has_isolated_nodes()}')
        print(f'Has self-loops: {data.has_self_loops()}')
        print(f'Is undirected: {data.is_undirected()}')
    
    return data, dataset

data, dataset = get_data(DATASET_NAME)


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [7]:
# # # restore np.load for future normal usage
# # np.load = old
# # del(old)
# np.load.__defaults__=(None, False, True, 'ASCII')

In [8]:
IgnoreStructure = False

if IgnoreStructure==True:
    nodeid=list(range(len(data.y)))
    data.edge_index=torch.tensor([nodeid,nodeid],dtype=torch.long)
    #print(data.edge_index)

### Dataset and Graph construction

[Cora dataset drawing](https://graphsandnetworks.com/the-cora-dataset/)

In [9]:
#create graph
import networkx as nx
import numpy as np
import torch
from torch_geometric.utils import to_networkx
from ipynb.fs.full.utils.GNNutils import save_gephi_graph

In [10]:
if DATASET_NAME in ['Moon','Fake']:
    G = to_networkx(data, to_undirected=True)
    
else:
    G_fillename=DIR+DATASET_NAME+".gpickle"

    if os.path.exists(G_fillename)==False:
        print("Graph is not found, creating it....")
        G = to_networkx(data, to_undirected=True)
        nx.write_gpickle(G, G_fillename)
        print("Done")
    else:
        print("Loading Saved graph...")
        G = nx.read_gpickle(G_fillename)
        print("Done")

Loading Saved graph...
Done


In [11]:
#save_gephi_graph(G, data.y, RESULTS_DIR+DATASET_NAME+'_original')

### Sparsification

In [12]:
methods=['ExactERSampling',
         'LocalERSampling',
         'SubmodularSparsificationLocal',
         'SubmodularSparsifierGlobal',
         'SpanningTree',
         'TSpanner',         
         'RandomGlobal',
         'RandomLocal',
         'LocalKNN',
         'SpectralSubmodularGlobal',
         'SpectralSubmodularLocal',
         'GSmain'
        ]

method_name = 'LocalKNN' 
STOverlay = False
IgnoreFeature = False

#### Available Optimizer

['random', 'modular', 'naive', 'lazy', 'approximate-lazy', 'two-stage', 'stochastic', 'sample', 'greedi', 'bidirectional', 'sieve']

#### Available distance metric

['euclidean', 'l2', 'l1', 'manhattan', 'cityblock', 'braycurtis', 'canberra', 'chebyshev', 'correlation', 'cosine', 'dice', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule', 'wminkowski', 'nan_euclidean', 'haversine'], or 'precomputed', or a callable

In [13]:
if IgnoreFeature==True:
    print(data.x.shape)
    p = torch.empty(data.x.shape).uniform_(0, 1)    
    x = torch.bernoulli(p)
    #x = torch.ones(data.x.shape)
    data.x = x

In [14]:
import time
sparse_start = time.time()

In [15]:
if method_name=='ExactERSampling':
    from ipynb.fs.full.SpectralSparsifier import  TrueERSparsify    
    
    sparserW, nz = TrueERSparsify(G, epsilon=0.90)
    H = nx.from_scipy_sparse_matrix(sparserW)
    
    ledge_index=np.array([sparserW.tocoo().row, sparserW.tocoo().col])
    edge_weight=sparserW.tocoo().data
    
elif method_name=='LocalERSampling':
    from ipynb.fs.full.SpectralSparsifier import  LocalERSparsify, LocalERfast    
    
    #sparserW, nz = LocalERSparsify(G, 0.70, computation='parallel', window=NUM_PROCESSORS*1000)
    sparserW, nz = LocalERSparsify(G, 0.70, computation='linear', window=NUM_PROCESSORS*1000)
    #sparserW, nz = LocalERfast(G, 0.70, computation='parallel', window=NUM_PROCESSORS*100)
    
    H = nx.from_scipy_sparse_matrix(sparserW)
    
    ledge_index=np.array([sparserW.tocoo().row, sparserW.tocoo().col])
    edge_weight=sparserW.tocoo().data
    
elif method_name=='SubmodularSparsificationLocal':
    from ipynb.fs.full.SubmodularSparsifierLocal import  LocalKSparsify, LocalKSparsifyParallel
    from ipynb.fs.full.SubmodularSparsifierLocal import LocalKSparsifyParallelCustom,LocalKSparsifyFacilityLocation
    
    H=LocalKSparsifyParallelCustom(G, data.x.numpy(), K=1, window=NUM_PROCESSORS*100, njobs=NUM_PROCESSORS)
    #H=LocalKSparsify(G, data.x.numpy(), K=1)
    #H=LocalKSparsifyFacilityLocation(G, data.x.numpy(), K=1,window=NUM_PROCESSORS*100, njobs=NUM_PROCESSORS,metric='cosine')
    
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
elif method_name=='SubmodularSparsifierGlobal':
    from ipynb.fs.full.SubmodularSparsifierGlobal import  GlobalSparsifier 
    
    H=GlobalSparsifier(G, data.x.numpy(), 10000, optimizer='stochastic')
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
elif method_name=='SpectralSubmodularGlobal':
    from ipynb.fs.full.SpectralSubmodularGlobal import  GlobalSubSpecSparsifier 
    
    H=GlobalSubSpecSparsifier(G, data.x.numpy(), 2500, optimizer='approximate-lazy')
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
elif method_name=='SpectralSubmodularLocal':
    from ipynb.fs.full.SpectralSubmodularLocal import  LocalKSparsify, LocalKSparsifySpectral
    
    H=LocalKSparsify(G, data.x.numpy(), K=2)
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
elif method_name=='GSmain':
    from ipynb.fs.full.GraphSparsiferMain import LocalKSparsifySpectral, SpectralSubmodularK 
    
    #H=LocalKSparsifySpectral(G, data.x.numpy(), K=2)
    H=SpectralSubmodularK(G, data.x.numpy(), K=3)
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
    
elif method_name=='SpanningTree':
    from ipynb.fs.full.SpanningTreeSparsifier import  STSparsify 
    
    H=STSparsify(G)
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
elif method_name=='TSpanner':
    from ipynb.fs.full.SpanningTreeSparsifier import  TSpannerSparsify 
    
    H=TSpannerSparsify(G, stretch=20)
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
elif method_name=='RandomGlobal':
    from ipynb.fs.full.SpanningTreeSparsifier import  RandomWieghtedSparsify 
    
    H=RandomWieghtedSparsify(G,0.95)
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
elif method_name=='RandomLocal':
    from ipynb.fs.full.SpanningTreeSparsifier import  RandomKLocal, RandomKLocalLinear
    
    H=RandomKLocalLinear(G, data.x.numpy(), K=1)
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
    
elif method_name=='LocalKNN':
    from ipynb.fs.full.SpanningTreeSparsifier import  LocalKNN, LocalKNNParallel
    
    H = LocalKNN(G, data.x.numpy(), K=2)
    #H = LocalKNNParallel(G, data.x.numpy(), K=5, window=NUM_PROCESSORS*100, njobs=NUM_PROCESSORS)
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
    
else:
    print("The sparsification algorithm is not defined")
    ledge_index=[[],[]]
    edge_weight=None
    H=nx.Graph()
    
print(len(ledge_index[0]))

Cpu count:  20


  0%|          | 0/2708 [00:00<?, ?it/s]

7186


In [16]:
sparse_end = time.time()
print ("Time elapsed:", sparse_end - sparse_start)

Time elapsed: 0.5909264087677002


In [17]:
#STOverlay = False

In [18]:
if STOverlay==True:
    from ipynb.fs.full.SpanningTreeSparsifier import  STSparsify 
        
    ST_H=STSparsify(G)    
    H = nx.compose(H,ST_H)
    
    edges=np.array(list(H.edges))
    ledge_index=[list(edges[:,0]),list(edges[:,1])]
    ledge_index[0].extend(list(edges[:,1]))
    ledge_index[1].extend(list(edges[:,0]))
    edge_weight=None
    
    print(len(ledge_index[0]))    

In [19]:
if DATASET_NAME in ['Moon','Fake']:
    import copy
    ldata=copy.deepcopy(data)
else:
    ldata=dataset[0]


if IgnoreFeature==True:
    ldata.x = x
    
if IgnoreStructure==True:
    nodeid=list(range(len(data.y)))
    ledge_index=[nodeid,nodeid]
    edge_weight=None

In [20]:
ldata.edge_index=torch.tensor(ledge_index, dtype=torch.long)

if edge_weight is None:
    print("Edge weights not updated")
else:
    ldata.edge_weight=torch.tensor(edge_weight, dtype=torch.float)

Edge weights not updated


In [21]:
# H_fillename=DIR+DATASET_NAME+"_sub_spectral.gpickle"
# nx.write_gpickle(H, H_fillename)

In [22]:
#save_gephi_graph(H, ldata.y, RESULTS_DIR+DATASET_NAME+"-"+method_name)

In [23]:
# nodes=list(G.nodes())

# for u in nodes[:100]:
#     V=list(G.neighbors(u))
#     print(data.y[u])
#     print(data.y[V])
#     print("-"*50)  
    
#     print("Sparse..")
    
#     V=list(H.neighbors(u))
#     print(data.y[u])
#     print(data.y[V])
#     print("*"*50)
    

In [24]:
if DATASET_NAME=='Moon':
    import ipynb.fs.full.utils.MoonGraph as GMoon        
#     GMoon.drawMoon(G,data)
#     GMoon.drawMoon(H,data)
    
    GMoon.draw_blobs_data(G,data)
    GMoon.draw_blobs_data(H,data)

#### Construct hybrid graph

In [25]:
print(ldata.edge_index.shape)
print(data.edge_index.shape)

torch.Size([2, 7186])
torch.Size([2, 10556])


### GNN performance

In [26]:
from ipynb.fs.full.GNNalgorithms import GCNperformance, GATperformance
from ipynb.fs.full.GCNGATsampler import GCNperformanceSampler, GATperformanceSampler
from ipynb.fs.full.GSAGE import GSAGEperformance
from ipynb.fs.full.GSAGEmultilabel import GSAGEperformance as GSAGEperformanceMultiLabel
from ipynb.fs.full.GSAINT import GSAINTperformance

There are 2 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
cuda
Cpu count:  20
Data directory:  /scratch/gilbreth/das90/Dataset/
Result directory: /scratch/gilbreth/das90/Dataset/RESULTS/
There are 2 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
cuda
Cpu count:  20
Data directory:  /scratch/gilbreth/das90/Dataset/
Result directory: /scratch/gilbreth/das90/Dataset/RESULTS/
There are 2 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
cuda
Cpu count:  20
Data directory:  /scratch/gilbreth/das90/Dataset/
Result directory: /scratch/gilbreth/das90/Dataset/RESULTS/
There are 2 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
cuda
Cpu count:  20
Data directory:  /scratch/gilbreth/das90/Dataset/
Result directory: /scratch/gilbreth/das90/Dataset/RESULTS/
There are 2 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
cuda
Cpu count:  20
Data directory:  /scratch/gilbreth/das90/Dataset/
Result directory: /scratch/gilbreth/das90/Dataset/RESU

In [27]:
# GSAINTperformance(data, dataset, epochs=30,train_method='Sample',inference_method='Sample',
#                   train_neighbors=[-1,10],
#                   test_neighbors=[-1,10])

In [28]:
# GSAINTperformance(ldata, dataset, epochs=30,train_method='Sample',inference_method='Sample',
#                   train_neighbors=[-1,-1],
#                   test_neighbors=[-1,-1])

In [29]:
#GSAGEperformance(data, dataset, epochs=50, train_neighbors=[25,10])

In [30]:
# GSAGEperformance(ldata, dataset, epochs=50, train_neighbors=[25, 10])

In [31]:
#GSAGEperformanceMultiLabel(data, dataset, epochs=20, train_neighbors=[25,10])

In [32]:
#GSAGEperformanceMultiLabel(ldata, dataset, epochs=20, train_neighbors=[25,10])

In [33]:
GCNperformance(data, dataset, epochs=100)

GCN(
  (conv1): GCNConv(1433, 256)
  (conv2): GCNConv(256, 7)
)
Epoch: 001, Loss: 1.9447, Val: 0.3720, Test: 0.3820
Epoch: 002, Loss: 1.9220, Val: 0.3740, Test: 0.3930
Epoch: 003, Loss: 1.8934, Val: 0.4200, Test: 0.4190
Epoch: 004, Loss: 1.8559, Val: 0.4200, Test: 0.4290
Epoch: 005, Loss: 1.8135, Val: 0.4660, Test: 0.4740
Epoch: 006, Loss: 1.7646, Val: 0.5240, Test: 0.5420
Epoch: 007, Loss: 1.7154, Val: 0.5960, Test: 0.6090
Epoch: 008, Loss: 1.6547, Val: 0.6560, Test: 0.6640
Epoch: 009, Loss: 1.5888, Val: 0.6880, Test: 0.7170
Epoch: 010, Loss: 1.5195, Val: 0.7120, Test: 0.7460
Epoch: 011, Loss: 1.4485, Val: 0.7400, Test: 0.7630
Epoch: 012, Loss: 1.3815, Val: 0.7580, Test: 0.7750
Epoch: 013, Loss: 1.2946, Val: 0.7720, Test: 0.7820
Epoch: 014, Loss: 1.2068, Val: 0.7720, Test: 0.7850
Epoch: 015, Loss: 1.1322, Val: 0.7840, Test: 0.7870
Epoch: 016, Loss: 1.0754, Val: 0.7840, Test: 0.7930
Epoch: 017, Loss: 0.9883, Val: 0.7840, Test: 0.8010
Epoch: 018, Loss: 0.9034, Val: 0.7860, Test: 0.8020


0.81

In [34]:
GCNperformance(ldata, dataset, epochs=100)

GCN(
  (conv1): GCNConv(1433, 256)
  (conv2): GCNConv(256, 7)
)
Epoch: 001, Loss: 1.9450, Val: 0.2200, Test: 0.2460
Epoch: 002, Loss: 1.9195, Val: 0.2200, Test: 0.2420
Epoch: 003, Loss: 1.8874, Val: 0.2420, Test: 0.2690
Epoch: 004, Loss: 1.8472, Val: 0.2900, Test: 0.3140
Epoch: 005, Loss: 1.7986, Val: 0.3720, Test: 0.4050
Epoch: 006, Loss: 1.7464, Val: 0.4460, Test: 0.4810
Epoch: 007, Loss: 1.6911, Val: 0.5340, Test: 0.5370
Epoch: 008, Loss: 1.6260, Val: 0.6180, Test: 0.6130
Epoch: 009, Loss: 1.5524, Val: 0.6800, Test: 0.6890
Epoch: 010, Loss: 1.4807, Val: 0.7080, Test: 0.7250
Epoch: 011, Loss: 1.3963, Val: 0.7400, Test: 0.7550
Epoch: 012, Loss: 1.3271, Val: 0.7580, Test: 0.7600
Epoch: 013, Loss: 1.2340, Val: 0.7680, Test: 0.7640
Epoch: 014, Loss: 1.1427, Val: 0.7680, Test: 0.7610
Epoch: 015, Loss: 1.0582, Val: 0.7720, Test: 0.7630
Epoch: 016, Loss: 0.9962, Val: 0.7820, Test: 0.7670
Epoch: 017, Loss: 0.9090, Val: 0.7920, Test: 0.7750
Epoch: 018, Loss: 0.8238, Val: 0.7960, Test: 0.7790


0.782

In [35]:
# GCNperformanceSampler(data, dataset, epochs=100, train_neighbors=[-1,10],test_neighbors=[-1,10])

In [36]:
# GCNperformanceSampler(ldata, dataset, epochs=100, train_neighbors=[-1,10],test_neighbors=[-1,10])

In [37]:
#ldata.edge_weight=None

In [38]:
#GATperformance(data, dataset, epochs=100)

In [39]:
#GATperformance(ldata, dataset, epochs=100)

In [40]:
#GATperformanceSampler(data, dataset, epochs=100, train_neighbors=[2,2,2],test_neighbors=[2,2,2])

In [41]:
#GATperformanceSampler(ldata, dataset, epochs=100, train_neighbors=[-1,-1,-1],test_neighbors=[-1,-1,-1])